
# Lab 9: Build a Log Aggregator

In this lab, you will create your own log generator, build a command-line utility that scans log files, summarizes their contents, and provides insight into system behavior. Data structures to track log message levels such as `INFO`, `WARNING`, `ERROR`, and `CRITICAL`.

This lab reinforces:
- File I/O
- Pattern recognition (regex)
- Dictionaries and counters
- Functions and modularity
- Optional: CLI arguments, logging



## Part 1: Create Log files (20%)
Using the the following example log format below create a **python file** that will log errors In a structured tree format 

You will find examples in the folder called Logs that you can use to build your program.

Remember set of logs should have a varied levels of log entries (`INFO`, `WARNING`, `ERROR`, `CRITICAL`) and tailored message types for different service components.
You must create 5 structured logs here are some examples:

    sqldb
    ui
    frontend.js
    backend.js
    frontend.flask
    backend.flask

You may use chat GPT to create sample outputs NOT THE LOGS. IE:

    System failure
    Database corruption
    Disk failure detected
    Database corruption


In [ ]:
# Paste your python file here 
# don't forget to upload it with your submission


### Example Log Format

You will work with logs that follow this simplified structure:

```
2025-04-11 23:20:36,913 | my_app | INFO | Request completed
2025-04-11 23:20:36,914 | my_app.utils | ERROR | Unhandled exception
2025-04-11 23:20:36,914 | my_app.utils.db | CRITICAL | Disk failure detected
```


## Part 2: Logging the Log File (40%)
    New File
### Part 2a: Read the Log File (see lab 7) (10%)


Write a function to read the contents of a log file into a list of lines. Handle file errors gracefully.

### Part 2b: Parse Log Lines (see code below if you get stuck) (10%)

Use a regular expression to extract:
- Timestamp
- Log name
- Log level
- Message

### Part 2c: Count Log Levels (20%)

Create a function to count how many times each log level appears. Store the results in a dictionary. Then output it as a Json File
You may pick your own format but here is an example. 
```python
{
    "INFO": 
    {
        "Request completed": 42, 
        "Heartbeat OK": 7
    }

    "WARNING":
    {
        ...
    }
}

```


In [ ]:
# Paste your python file here don't for get to upload it with your submission

In [ ]:
# Paste your python file here 
# don't forget to upload it with your submission


## Step 3: Generate Summary Report (40%)
    New File
### Step 3a (20%):
 Develop a function that continuously monitors your JSON file(s) and will print a real-time summary of log activity. It should keep count of the messages grouped by log level (INFO, WARNING, ERROR, CRITICAL) and display only the critical messages. (I.e. If new data comes in the summary will change and a new critical message will be printed)
 - note: do not reprocess the entire file on each update.  

### Step 3a: Use a Matplotlib (Lecture 10) (20%)
Develop a function that continuously monitors your JSON file(s) and will graph in real-time a bar or pie plot of each of the errors.  (a graph for each log level). 
- The graph should show the distribution of log messages by level  (INFO, WARNING, ERROR, CRITICAL)  


### Critical notes:
- Your code mus use Daemon Threads (Lecture 14)
- 3a and 3b do not need to run at the same time. 


In [2]:
import json
import os
import queue
import sys
import threading
import time
from collections import Counter
from pathlib import Path
from typing import Callable, Dict, List

#LOGS

In [1]:
import logging
import logging.handlers

#Format
formatter = logging.Formatter(
    fmt="%(asctime)s | %(name)s | %(levelname)s | %(message)s"
)

# Parent logger
parentLog = logging.getLogger("sys")
parentLog.setLevel(logging.DEBUG)

# Utility function to attach handler to a logger
def setup_logger(name, level, filename):
    logger = logging.getLogger(name)
    logger.setLevel(level)
    handler = logging.handlers.TimedRotatingFileHandler(
        filename=filename,
        when="D",
        backupCount=2,
    )
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.propagate = False  # Prevent logging to parent
    return logger

# Set up each logger with its own file
infoLog = setup_logger("sys.info", logging.INFO, "info.log")
warningLog = setup_logger("sys.warning", logging.WARNING, "warning.log")
errorLog = setup_logger("sys.error", logging.ERROR, "error.log")
criticalLog = setup_logger("sys.critical", logging.CRITICAL, "critical.log")

# Example usage
infoLog.info("This is an info message.")
warningLog.warning("This is a warning.")
errorLog.error("This is an error.")
criticalLog.critical("This is critical.")


## SUmmary

In [3]:
def parse_log_line(line: str) -> dict | None:
    pattern = r"^(.*?)\s\|\s([\w\\.]+)\s\|\s(\w+)\s\|\s(.*)$"
    match = re.match(pattern, line)
    if match:
        ts, logger, level, msg = match.groups()
        return {
            "asctime": ts.strip(),
            "name": logger.strip(),
            "levelname": level.strip().upper(),
            "message": msg.strip()
        }
    return None

In [4]:
class LogFileTailer(threading.Thread):
    """Tail a single plain-text log file in real time and push parsed records."""

    def __init__(self, path: Path, out_q: "queue.Queue[dict]", *, poll: float = 0.4):
        super().__init__(daemon=True)
        self.path = path
        self.poll = poll
        self.out_q = out_q

In [6]:

def run(self) -> None:
    with self.path.open("r", encoding="utf-8") as f:
        f.seek(0, os.SEEK_END)
        while True:
            line = f.readline()
            if line:
                rec = parse_log_line(line)
                if rec:
                    self.out_q.put(rec)
            else:
                time.sleep(self.poll)

In [8]:
class SummaryPrinter(threading.Thread):
    """Print running counts and any new CRITICAL messages."""

    def __init__(self, in_q: "queue.Queue[dict]", *, poll: float = 0.2):
        super().__init__(daemon=True)
        self.in_q = in_q
        self.poll = poll
        self.counts = Counter({lvl: 0 for lvl in LOG_LEVELS})
        self.lock = threading.Lock()

    def snapshot_counts(self) -> Dict[str, int]:
        with self.lock:
            return dict(self.counts)

    def run(self) -> None:
        while True:
            try:
                rec = self.in_q.get(timeout=self.poll)
            except queue.Empty:
                continue
            level = rec.get("levelname")
            if level in LOG_LEVELS:
                with self.lock:
                    self.counts[level] += 1
                if level == "CRITICAL":
                    ts = rec.get("asctime", "<no time>")
                    msg = rec.get("message", "<no message>")
                    print(f"\n!!! CRITICAL [{ts}] {msg}")
            self._print_summary()

    def _print_summary(self) -> None:
        summary = " | ".join(f"{lvl}: {self.counts[lvl]}" for lvl in LOG_LEVELS)
        print(f"\r[Counts] {summary}", end="", flush=True)


In [9]:
class LiveBarPlot(threading.Thread):
    """Update a bar chart in real time based on the SummaryPrinter counts."""

    def __init__(self, summary: SummaryPrinter, interval_ms: int = 800):
        super().__init__(daemon=True)
        self.summary = summary
        self.interval_ms = interval_ms

    def run(self) -> None:
        plt.ion()
        self.fig, self.ax = plt.subplots()
        self.bars = self.ax.bar(LOG_LEVELS, [0] * len(LOG_LEVELS))
        self.ax.set_ylim(0, 1)
        self.ax.set_ylabel("Count")
        self.ax.set_title("Live Log-Level Distribution")

        def _update(_frame):
            counts = self.summary.snapshot_counts()
            max_val = max(counts.values()) or 1
            self.ax.set_ylim(0, max_val * 1.1)
            for bar, lvl in zip(self.bars, LOG_LEVELS):
                bar.set_height(counts[lvl])
            return self.bars

        self.anim = FuncAnimation(self.fig, _update, interval=self.interval_ms)
        plt.show(block=True)


In [ ]:
def main(paths: List[str]):
    if not paths:
        print("Usage: python log_monitor.py <log1.txt> [log2.txt …]", file=sys.stderr)
        sys.exit(1)

    record_q: "queue.Queue[dict]" = queue.Queue()

    tailers = [LogFileTailer(Path(p), record_q) for p in paths]
    for t in tailers:
        t.start()
        print(f"Tailing {t.path} …")

    summary = SummaryPrinter(record_q)
    summary.start()

    plotter = LiveBarPlot(summary)
    plotter.start()

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("\nStopping log monitor – goodbye!")


if __name__ == "__main__":
    main(sys.argv[1:])

In [ ]:
# Here is a sample regex that parses a log file and extracts relevant information. 
# you will need to modify it. Review Lecture 11
import re

def parse_log_line(line):
    pattern = r"^(.*?)\s\|\s(\w+)\s\|\s(\w+)\s\|\s(.*)$"
    match = re.match(pattern, line)
   
